In [1]:
import os
import conda
conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(conda_dir, 'Library\share')
# proj_lib = os.path.join(os.path.join(conda_dir, 'pkgs'), 'proj4-5.2.0-h6538335_1006\Library\share')
path_gdal = os.path.join(proj_lib, 'gdal')
os.environ ['PROJ_LIB']=proj_lib
os.environ ['GDAL_DATA']=path_gdal

from datetime import datetime
import pandas as pd
import geopandas as gpd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

import shapely
from shapely.geometry import Point, LineString, MultiLineString, Polygon

In [2]:
gdf_bui = gpd.read_file('buildings_1_СТН.json',encoding='utf-8')

In [3]:
gdf_addr = gpd.read_file('../geocoding/gdf_ok.json',encoding='utf-8')

In [46]:
gdf_addr = gdf_addr.rename(columns={'address':'addr_ref'})
try:
    del gdf_addr['index_right']
except:
    pass

In [79]:
def connect_addr_to_bui(gdf_addr,gdf_bui,buff):
    bui_buff = gdf_bui.copy()
    bui_buff.crs='epsg:4326'
    bui_buff.geometry = bui_buff.geometry.to_crs('epsg:32637').buffer(buff).to_crs('epsg:4326')
    #
    sj_poly = gpd.sjoin(bui_buff[['osm_id','geometry']], gdf_addr).drop('index_right',axis=1)
    
    del sj_poly['geometry']
    
    bui_new = sj_poly.merge(gdf_bui,how='left',on=['osm_id'])
    bui_new = gpd.GeoDataFrame(bui_new)
    bui_new.crs='epsg:4326'

    return bui_new

In [76]:
zero_buff = connect_addr_to_bui(gdf_addr,gdf_bui,0)

In [52]:
not_found_addr_zero = gdf_addr[~gdf_addr.addr_ref.isin(zero_buff.addr_ref)].reset_index(drop=True)

In [53]:
not_found_bui_zero = gdf_bui[~gdf_bui.osm_id.isin(zero_buff.osm_id)].reset_index(drop=True)

In [54]:
ten_buff = connect_addr_to_bui(not_found_addr_zero,not_found_bui_zero,10)

In [60]:
not_found_addr_ten = gdf_addr[( (~gdf_addr.addr_ref.isin(zero_buff.addr_ref) 
                               & (~gdf_addr.addr_ref.isin(ten_buff.addr_ref))))
                             ].reset_index(drop=True)

In [61]:
not_found_bui_ten = gdf_bui[((~gdf_bui.osm_id.isin(zero_buff.osm_id)) 
                             & (~gdf_bui.osm_id.isin(ten_buff.osm_id))
                            )].reset_index(drop=True)

In [62]:
thirty_buff = connect_addr_to_bui(not_found_addr_ten,not_found_bui_ten,30)

In [94]:
zero_buff = zero_buff.drop_duplicates(subset=['osm_id']).reset_index(drop=True)
ten_buff = ten_buff.drop_duplicates(subset=['osm_id']).reset_index(drop=True)
thirty_buff = thirty_buff.drop_duplicates(subset=['osm_id']).reset_index(drop=True)

In [ ]:
zero_buff['buff'] = 0
ten_buff['buff'] = 10
thirty_buff['buff'] = 30

In [108]:
all_found = zero_buff.append(ten_buff).append(thirty_buff).reset_index(drop=True)

In [110]:
all_found.to_file('all_found.json',driver='GeoJSON',encoding='utf-8')

In [114]:
all_bui = gdf_bui[~gdf_bui.osm_id.isin(all_found.osm_id)].reset_index(drop=True)

In [116]:
lst_all_clmn = list(all_found.columns)

i=0
for clmn in lst_all_clmn:
    if clmn not in gdf_bui.columns:
        all_bui[clmn] = None
    #
#

In [118]:
all_bui = all_bui[lst_all_clmn]

In [120]:
all_bui = all_bui.append(all_found).reset_index(drop=True)

In [121]:
all_bui.to_file('all_bui.json',driver='GeoJSON',encoding='utf-8')